<a href="https://colab.research.google.com/github/byunsy/bioinformatics-algorithms-py/blob/main/BA_4G_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Leaderboard Cyclopeptide Sequencing Problem

### Function

In [1]:
def Expand(pep):

    amino_acid = ['57', '71', '87', '97', '99', '101', '103', '113', '114', '115', '128', '129', '131', '137', '147', '156', '163', '186']
    new_pep = []

    for i in pep:
        for amino in amino_acid:
            if i != "":
                new_pep.append(i+"-"+amino)
            else:
                new_pep.append(i+amino)

    return new_pep

In [2]:
def Mass(peptides):

    split_pep = peptides.split("-")
    mass = 0

    for i in split_pep:
        mass += int(i)

    return mass

In [3]:
def LinearSpectrum(peptide):
    
    split_pep = peptide.split("-")

    prefix_mass = [0]
    for amino in split_pep:
        prefix_mass.append(prefix_mass[-1] + int(amino))

    linear_spectrum = [0]
    for i in range(len(split_pep)):
        j = i + 1
        while j <= len(split_pep):
            linear_spectrum.append(prefix_mass[j]-prefix_mass[i])
            j += 1
    
    return sorted(linear_spectrum)

In [27]:
LinearSpectrum("114-128-129-113", amino_acid_mass)

[0, 113, 114, 128, 129, 242, 242, 257, 370, 371, 484]

In [4]:
from collections import Counter

def LinearScore(peptide, spectrum):

    pep_spectrum = LinearSpectrum(peptide)

    pep_counter = Counter(pep_spectrum) 
    spc_counter = Counter(spectrum)
    score = 0

    for i in set(pep_spectrum):
        if i in spectrum:
            if pep_counter[i] <= spc_counter[i]:
                score += pep_counter[i]
            else: # pep_counter[i] > spc_counter[i]
                score += spc_counter[i]

    return score

In [30]:
spectrum = [0, 99, 113, 114, 128, 227, 257, 299, 355, 356, 370, 371, 484]
print(LinearScore("114-128-129-113", spectrum))

8


In [5]:
def CyclicSpectrum(peptide):
    
    split_pep = peptide.split("-")

    prefix_mass = [0]
    for amino in split_pep:
        prefix_mass.append(prefix_mass[-1] + int(amino))

    peptide_mass = prefix_mass[-1]
    cyclic_spectrum = [0]
    for i in range(len(split_pep)):
        j = i + 1
        while j <= len(split_pep):
            cyclic_spectrum.append(prefix_mass[j]-prefix_mass[i])
            if i > 0 and j < len(split_pep):
                cyclic_spectrum.append(peptide_mass - (prefix_mass[j]-prefix_mass[i]))
            j += 1
    
    return sorted(cyclic_spectrum)

In [37]:
CyclicSpectrum("114-128-129-113")

[0, 113, 114, 128, 129, 227, 242, 242, 257, 355, 356, 370, 371, 484]

In [6]:
from collections import Counter

def CyclopeptideScore(peptide, spectrum):

    pep_spectrum = CyclicSpectrum(peptide)

    pep_counter = Counter(pep_spectrum) 
    spc_counter = Counter(spectrum)
    score = 0

    for i in set(pep_spectrum):
        if i in spectrum:
            if pep_counter[i] <= spc_counter[i]:
                score += pep_counter[i]
            else: # pep_counter[i] > spc_counter[i]
                score += spc_counter[i]

    return score

In [39]:
spectrum = [0, 99, 113, 114, 128, 227, 257, 299, 355, 356, 370, 371, 484]
print(CyclopeptideScore("114-128-129-113", spectrum))

11


In [7]:
def Trim(leaderboard, spectrum, n):

    linear_scores = []
    for pep in leaderboard:
        linear_scores.append(LinearScore(pep, spectrum))

    lb_pep    = [p for _,p in sorted(zip(linear_scores,leaderboard), reverse=True)]
    lb_scores = [s for s,_ in sorted(zip(linear_scores,leaderboard), reverse=True)]

    cutoff_idx = n

    for i in range(cutoff_idx, len(lb_scores)):
        if lb_scores[i] < lb_scores[i-1]:
            break
        else:
            cutoff_idx += 1

    return lb_pep[:cutoff_idx]

In [8]:
def LBCyclopeptideSequencing(spectrum, n):
    
    parent_mass = spectrum[-1]
    leaderboard = [""]
    leader_pep = "0"

    while len(leaderboard) != 0:
        leaderboard = Expand(leaderboard)

        to_remove = set()
        for pep in leaderboard:
            if Mass(pep) == parent_mass:
                if CyclopeptideScore(pep, spectrum) > CyclopeptideScore(leader_pep, spectrum):
                    leader_pep = pep
            elif Mass(pep) > parent_mass:
                to_remove.add(pep)

        for i in to_remove:
            leaderboard.remove(i)

        leaderboard = Trim(leaderboard, spectrum, n)

    return leader_pep

### Test Cases

In [11]:
# Create a function for test suite
def TestSuite(function, cases):
    print("*"*50)
    print("TEST SUITE\n")
    passed = 0
    for i, case in enumerate(cases):
        spectrum, n, answer = case
        result = function(spectrum, n)
        if result == answer:
            print("- Test Case {} Passed. Expected: {}, Actual: {}"
                  .format(i+1, answer, result))
            passed += 1
        else:
            print("- Test Case {} Failed. Expected: {}, Actual: {}"
                  .format(i+1, answer, result))
    print("\n{} out of {} passed.".format(passed, len(cases)), end=" ")
    print("END OF TEST SUITE.")
    print("*"*50)

In [12]:
# Create test cases to pass into test suite
case1 = ([0, 71, 113, 129, 147, 200, 218, 260, 313, 331, 347, 389, 460], 10, "113-147-71-129")

cases = [case1]

TestSuite(LBCyclopeptideSequencing, cases)

**************************************************
TEST SUITE

- Test Case 1 Failed. Expected: 113-147-71-129, Actual: 71-147-113-129

0 out of 1 passed. END OF TEST SUITE.
**************************************************


**Note**. Multiple solutions may exist. Despite not passing the test suite above, the auto grader in Stepik confirms the algorithm is working appropriately. 

### Stepik Coding Exercise

In [ ]:
sp1 = [0, 71, 113, 129, 147, 200, 218, 260, 313, 331, 347, 389, 460]

print(LBCyclopeptideSequencing(sp1, 10))

113-147-71-129


In [ ]:
sp2 = [0, 71, 71, 71, 87, 97, 97, 99, 101, 103, 113, 113, 114, 115, 128, 128, 129, 137, 147, 163, 163, 170, 184, 184, 186, 186, 190, 211, 215, 226, 226, 229, 231, 238, 241, 244, 246, 257, 257, 276, 277, 278, 299, 300, 312, 316, 317, 318, 318, 323, 328, 340, 343, 344, 347, 349, 356, 366, 370, 373, 374, 391, 401, 414, 414, 415, 419, 427, 427, 431, 437, 441, 446, 453, 462, 462, 462, 470, 472, 502, 503, 503, 511, 515, 529, 530, 533, 533, 540, 543, 547, 556, 559, 569, 574, 575, 584, 590, 600, 600, 604, 612, 616, 617, 630, 640, 640, 643, 646, 648, 660, 671, 683, 684, 687, 693, 703, 703, 719, 719, 719, 729, 730, 731, 737, 740, 741, 745, 747, 754, 774, 780, 784, 790, 797, 800, 806, 818, 826, 827, 832, 833, 838, 846, 846, 847, 850, 868, 869, 877, 884, 889, 893, 897, 903, 908, 913, 917, 930, 940, 947, 956, 960, 960, 961, 964, 965, 966, 983, 983, 985, 1002, 1009, 1010, 1011, 1021, 1031, 1031, 1036, 1053, 1054, 1058, 1059, 1062, 1063, 1074, 1076, 1084, 1092, 1103, 1113, 1122, 1124, 1130, 1133, 1134, 1145, 1146, 1146, 1149, 1150, 1155, 1156, 1171, 1173, 1174, 1187, 1191, 1193, 1200, 1212, 1221, 1233, 1240, 1242, 1246, 1259, 1260, 1262, 1277, 1278, 1283, 1284, 1287, 1287, 1288, 1299, 1300, 1303, 1309, 1311, 1320, 1330, 1341, 1349, 1357, 1359, 1370, 1371, 1374, 1375, 1379, 1380, 1397, 1402, 1402, 1412, 1422, 1423, 1424, 1431, 1448, 1450, 1450, 1467, 1468, 1469, 1472, 1473, 1473, 1477, 1486, 1493, 1503, 1516, 1520, 1525, 1530, 1536, 1540, 1544, 1549, 1556, 1564, 1565, 1583, 1586, 1587, 1587, 1595, 1600, 1601, 1606, 1607, 1615, 1627, 1633, 1636, 1643, 1649, 1653, 1659, 1679, 1686, 1688, 1692, 1693, 1696, 1702, 1703, 1704, 1714, 1714, 1714, 1730, 1730, 1740, 1746, 1749, 1750, 1762, 1773, 1785, 1787, 1790, 1793, 1793, 1803, 1816, 1817, 1821, 1829, 1833, 1833, 1843, 1849, 1858, 1859, 1864, 1877, 1886, 1890, 1893, 1900, 1900, 1903, 1904, 1918, 1922, 1930, 1930, 1931, 1961, 1963, 1971, 1971, 1971, 1980, 1987, 1992, 1996, 2002, 2006, 2006, 2014, 2018, 2019, 2019, 2032, 2042, 2059, 2060, 2063, 2067, 2077, 2084, 2086, 2089, 2090, 2093, 2105, 2110, 2115, 2115, 2116, 2117, 2121, 2133, 2134, 2155, 2156, 2157, 2176, 2176, 2187, 2189, 2192, 2195, 2202, 2204, 2207, 2207, 2218, 2222, 2243, 2247, 2247, 2249, 2249, 2263, 2270, 2270, 2286, 2296, 2304, 2305, 2305, 2318, 2319, 2320, 2320, 2330, 2332, 2334, 2336, 2336, 2346, 2362, 2362, 2362, 2433]

print(LBCyclopeptideSequencing(sp2, 325))

163-114-97-129-97-147-99-71-186-71-113-163-115-71-113-128-103-87-128-101-137


In [ ]:
string = "0 71 71 71 97 103 113 113 115 115 128 128 128 129 129 129 131 142 156 163 174 184 186 186 186 226 227 228 232 234 243 244 246 255 257 259 276 283 284 298 299 299 303 314 314 315 315 329 337 347 355 356 370 374 375 400 411 411 412 413 426 427 427 429 430 443 444 445 450 462 466 484 503 515 526 533 539 540 542 542 542 555 559 560 561 563 572 579 586 590 599 613 636 643 654 654 655 657 661 668 670 671 676 684 687 689 689 690 705 714 726 728 741 749 764 765 774 776 782 783 786 786 797 799 818 818 818 840 843 845 854 862 862 870 875 877 879 889 889 889 893 911 914 915 946 953 960 960 960 968 969 972 974 983 985 990 990 992 1004 1008 1008 1018 1031 1045 1048 1063 1081 1089 1097 1097 1098 1098 1101 1101 1102 1105 1116 1116 1121 1121 1123 1132 1134 1173 1176 1176 1187 1192 1204 1210 1213 1218 1226 1229 1229 1230 1236 1244 1244 1252 1263 1275 1283 1288 1289 1290 1291 1304 1307 1315 1333 1341 1344 1357 1358 1359 1360 1365 1373 1385 1396 1404 1404 1412 1418 1419 1419 1422 1430 1435 1438 1444 1456 1461 1472 1472 1475 1514 1516 1525 1527 1527 1532 1532 1543 1546 1547 1547 1550 1550 1551 1551 1559 1567 1585 1600 1603 1617 1630 1640 1640 1644 1656 1658 1658 1663 1665 1674 1676 1679 1680 1688 1688 1688 1695 1702 1733 1734 1737 1755 1759 1759 1759 1769 1771 1773 1778 1786 1786 1794 1803 1805 1808 1830 1830 1830 1849 1851 1862 1862 1865 1866 1872 1874 1883 1884 1899 1907 1920 1922 1934 1943 1958 1959 1959 1961 1964 1972 1977 1978 1980 1987 1991 1993 1994 1994 2005 2012 2035 2049 2058 2062 2069 2076 2085 2087 2088 2089 2093 2106 2106 2106 2108 2109 2115 2122 2133 2145 2150 2164 2182 2186 2198 2203 2204 2205 2218 2219 2221 2221 2222 2235 2236 2237 2237 2248 2273 2274 2278 2292 2293 2301 2311 2319 2333 2333 2334 2334 2345 2349 2349 2350 2364 2365 2372 2389 2391 2393 2402 2404 2405 2414 2416 2420 2421 2422 2462 2462 2462 2464 2474 2485 2492 2506 2517 2519 2519 2519 2520 2520 2520 2533 2533 2535 2535 2545 2551 2577 2577 2577 2648"

ret = string.replace(" ", ", ")
print(ret)

0, 71, 71, 71, 97, 103, 113, 113, 115, 115, 128, 128, 128, 129, 129, 129, 131, 142, 156, 163, 174, 184, 186, 186, 186, 226, 227, 228, 232, 234, 243, 244, 246, 255, 257, 259, 276, 283, 284, 298, 299, 299, 303, 314, 314, 315, 315, 329, 337, 347, 355, 356, 370, 374, 375, 400, 411, 411, 412, 413, 426, 427, 427, 429, 430, 443, 444, 445, 450, 462, 466, 484, 503, 515, 526, 533, 539, 540, 542, 542, 542, 555, 559, 560, 561, 563, 572, 579, 586, 590, 599, 613, 636, 643, 654, 654, 655, 657, 661, 668, 670, 671, 676, 684, 687, 689, 689, 690, 705, 714, 726, 728, 741, 749, 764, 765, 774, 776, 782, 783, 786, 786, 797, 799, 818, 818, 818, 840, 843, 845, 854, 862, 862, 870, 875, 877, 879, 889, 889, 889, 893, 911, 914, 915, 946, 953, 960, 960, 960, 968, 969, 972, 974, 983, 985, 990, 990, 992, 1004, 1008, 1008, 1018, 1031, 1045, 1048, 1063, 1081, 1089, 1097, 1097, 1098, 1098, 1101, 1101, 1102, 1105, 1116, 1116, 1121, 1121, 1123, 1132, 1134, 1173, 1176, 1176, 1187, 1192, 1204, 1210, 1213, 1218, 1226, 1229, 

In [ ]:
sp3 = [0, 71, 71, 71, 97, 103, 113, 113, 115, 115, 128, 128, 128, 129, 129, 129, 131, 142, 156, 163, 174, 184, 186, 186, 186, 226, 227, 228, 232, 234, 243, 244, 246, 255, 257, 259, 276, 283, 284, 298, 299, 299, 303, 314, 314, 315, 315, 329, 337, 347, 355, 356, 370, 374, 375, 400, 411, 411, 412, 413, 426, 427, 427, 429, 430, 443, 444, 445, 450, 462, 466, 484, 503, 515, 526, 533, 539, 540, 542, 542, 542, 555, 559, 560, 561, 563, 572, 579, 586, 590, 599, 613, 636, 643, 654, 654, 655, 657, 661, 668, 670, 671, 676, 684, 687, 689, 689, 690, 705, 714, 726, 728, 741, 749, 764, 765, 774, 776, 782, 783, 786, 786, 797, 799, 818, 818, 818, 840, 843, 845, 854, 862, 862, 870, 875, 877, 879, 889, 889, 889, 893, 911, 914, 915, 946, 953, 960, 960, 960, 968, 969, 972, 974, 983, 985, 990, 990, 992, 1004, 1008, 1008, 1018, 1031, 1045, 1048, 1063, 1081, 1089, 1097, 1097, 1098, 1098, 1101, 1101, 1102, 1105, 1116, 1116, 1121, 1121, 1123, 1132, 1134, 1173, 1176, 1176, 1187, 1192, 1204, 1210, 1213, 1218, 1226, 1229, 1229, 1230, 1236, 1244, 1244, 1252, 1263, 1275, 1283, 1288, 1289, 1290, 1291, 1304, 1307, 1315, 1333, 1341, 1344, 1357, 1358, 1359, 1360, 1365, 1373, 1385, 1396, 1404, 1404, 1412, 1418, 1419, 1419, 1422, 1430, 1435, 1438, 1444, 1456, 1461, 1472, 1472, 1475, 1514, 1516, 1525, 1527, 1527, 1532, 1532, 1543, 1546, 1547, 1547, 1550, 1550, 1551, 1551, 1559, 1567, 1585, 1600, 1603, 1617, 1630, 1640, 1640, 1644, 1656, 1658, 1658, 1663, 1665, 1674, 1676, 1679, 1680, 1688, 1688, 1688, 1695, 1702, 1733, 1734, 1737, 1755, 1759, 1759, 1759, 1769, 1771, 1773, 1778, 1786, 1786, 1794, 1803, 1805, 1808, 1830, 1830, 1830, 1849, 1851, 1862, 1862, 1865, 1866, 1872, 1874, 1883, 1884, 1899, 1907, 1920, 1922, 1934, 1943, 1958, 1959, 1959, 1961, 1964, 1972, 1977, 1978, 1980, 1987, 1991, 1993, 1994, 1994, 2005, 2012, 2035, 2049, 2058, 2062, 2069, 2076, 2085, 2087, 2088, 2089, 2093, 2106, 2106, 2106, 2108, 2109, 2115, 2122, 2133, 2145, 2150, 2164, 2182, 2186, 2198, 2203, 2204, 2205, 2218, 2219, 2221, 2221, 2222, 2235, 2236, 2237, 2237, 2248, 2273, 2274, 2278, 2292, 2293, 2301, 2311, 2319, 2333, 2333, 2334, 2334, 2345, 2349, 2349, 2350, 2364, 2365, 2372, 2389, 2391, 2393, 2402, 2404, 2405, 2414, 2416, 2420, 2421, 2422, 2462, 2462, 2462, 2464, 2474, 2485, 2492, 2506, 2517, 2519, 2519, 2519, 2520, 2520, 2520, 2533, 2533, 2535, 2535, 2545, 2551, 2577, 2577, 2577, 2648]

print(LBCyclopeptideSequencing(sp3, 243))

186-113-163-71-103-129-97-186-128-131-115-129-186-129-128-156-71-71-113-115-128


In [16]:
sp4 = [0, 71, 87, 97, 99, 103, 113, 114, 114, 128, 128, 128, 128, 129, 131, 131, 131, 131, 147, 156, 158, 163, 163, 202, 217, 227, 228, 234, 241, 242, 242, 243, 244, 256, 259, 259, 259, 262, 270, 277, 278, 289, 292, 294, 314, 330, 348, 355, 356, 357, 358, 370, 373, 375, 375, 376, 381, 387, 390, 405, 405, 417, 422, 423, 428, 429, 445, 460, 478, 484, 484, 495, 503, 504, 504, 504, 506, 516, 518, 518, 521, 531, 533, 536, 550, 551, 559, 573, 591, 592, 592, 609, 615, 617, 631, 632, 634, 635, 647, 647, 651, 662, 662, 664, 664, 672, 679, 679, 681, 687, 706, 723, 737, 738, 745, 746, 748, 762, 762, 763, 778, 778, 786, 790, 792, 792, 793, 793, 809, 810, 810, 834, 835, 835, 851, 865, 874, 876, 879, 889, 893, 906, 906, 907, 909, 920, 921, 921, 923, 925, 940, 941, 947, 949, 949, 966, 979, 996, 1005, 1007, 1020, 1020, 1034, 1034, 1037, 1037, 1037, 1040, 1051, 1052, 1054, 1056, 1063, 1068, 1069, 1076, 1077, 1094, 1110, 1135, 1148, 1159, 1163, 1165, 1165, 1165, 1166, 1167, 1167, 1168, 1168, 1168, 1168, 1183, 1185, 1190, 1191, 1197, 1222, 1239, 1262, 1264, 1266, 1273, 1288, 1293, 1294, 1295, 1296, 1296, 1296, 1296, 1297, 1298, 1299, 1304, 1319, 1326, 1328, 1330, 1353, 1370, 1395, 1401, 1402, 1407, 1409, 1424, 1424, 1424, 1424, 1425, 1425, 1426, 1427, 1427, 1427, 1429, 1433, 1444, 1457, 1482, 1498, 1515, 1516, 1523, 1524, 1529, 1536, 1538, 1540, 1541, 1552, 1555, 1555, 1555, 1558, 1558, 1572, 1572, 1585, 1587, 1596, 1613, 1626, 1643, 1643, 1645, 1651, 1652, 1667, 1669, 1671, 1671, 1672, 1683, 1685, 1686, 1686, 1699, 1703, 1713, 1716, 1718, 1727, 1741, 1757, 1757, 1758, 1782, 1782, 1783, 1799, 1799, 1800, 1800, 1802, 1806, 1814, 1814, 1829, 1830, 1830, 1844, 1846, 1847, 1854, 1855, 1869, 1886, 1905, 1911, 1913, 1913, 1920, 1928, 1928, 1930, 1930, 1941, 1945, 1945, 1957, 1958, 1960, 1961, 1975, 1977, 1983, 2000, 2000, 2001, 2019, 2033, 2041, 2042, 2056, 2059, 2061, 2071, 2074, 2074, 2076, 2086, 2088, 2088, 2088, 2089, 2097, 2108, 2108, 2114, 2132, 2147, 2163, 2164, 2169, 2170, 2175, 2187, 2187, 2202, 2205, 2205, 2211, 2216, 2217, 2217, 2219, 2222, 2234, 2235, 2236, 2237, 2244, 2262, 2278, 2298, 2300, 2303, 2314, 2315, 2322, 2330, 2333, 2333, 2333, 2336, 2348, 2349, 2350, 2350, 2351, 2358, 2364, 2365, 2375, 2390, 2429, 2429, 2434, 2436, 2445, 2461, 2461, 2461, 2461, 2463, 2464, 2464, 2464, 2464, 2478, 2478, 2479, 2489, 2493, 2495, 2505, 2521, 2592]


print(LBCyclopeptideSequencing(sp4, 217))

97-147-131-103-114-156-87-71-131-128-99-163-114-128-113-129-163-131-128-128-131


In [14]:
string = "0 71 87 97 99 103 113 114 114 128 128 128 128 129 131 131 131 131 147 156 158 163 163 202 217 227 228 234 241 242 242 243 244 256 259 259 259 262 270 277 278 289 292 294 314 330 348 355 356 357 358 370 373 375 375 376 381 387 390 405 405 417 422 423 428 429 445 460 478 484 484 495 503 504 504 504 506 516 518 518 521 531 533 536 550 551 559 573 591 592 592 609 615 617 631 632 634 635 647 647 651 662 662 664 664 672 679 679 681 687 706 723 737 738 745 746 748 762 762 763 778 778 786 790 792 792 793 793 809 810 810 834 835 835 851 865 874 876 879 889 893 906 906 907 909 920 921 921 923 925 940 941 947 949 949 966 979 996 1005 1007 1020 1020 1034 1034 1037 1037 1037 1040 1051 1052 1054 1056 1063 1068 1069 1076 1077 1094 1110 1135 1148 1159 1163 1165 1165 1165 1166 1167 1167 1168 1168 1168 1168 1183 1185 1190 1191 1197 1222 1239 1262 1264 1266 1273 1288 1293 1294 1295 1296 1296 1296 1296 1297 1298 1299 1304 1319 1326 1328 1330 1353 1370 1395 1401 1402 1407 1409 1424 1424 1424 1424 1425 1425 1426 1427 1427 1427 1429 1433 1444 1457 1482 1498 1515 1516 1523 1524 1529 1536 1538 1540 1541 1552 1555 1555 1555 1558 1558 1572 1572 1585 1587 1596 1613 1626 1643 1643 1645 1651 1652 1667 1669 1671 1671 1672 1683 1685 1686 1686 1699 1703 1713 1716 1718 1727 1741 1757 1757 1758 1782 1782 1783 1799 1799 1800 1800 1802 1806 1814 1814 1829 1830 1830 1844 1846 1847 1854 1855 1869 1886 1905 1911 1913 1913 1920 1928 1928 1930 1930 1941 1945 1945 1957 1958 1960 1961 1975 1977 1983 2000 2000 2001 2019 2033 2041 2042 2056 2059 2061 2071 2074 2074 2076 2086 2088 2088 2088 2089 2097 2108 2108 2114 2132 2147 2163 2164 2169 2170 2175 2187 2187 2202 2205 2205 2211 2216 2217 2217 2219 2222 2234 2235 2236 2237 2244 2262 2278 2298 2300 2303 2314 2315 2322 2330 2333 2333 2333 2336 2348 2349 2350 2350 2351 2358 2364 2365 2375 2390 2429 2429 2434 2436 2445 2461 2461 2461 2461 2463 2464 2464 2464 2464 2478 2478 2479 2489 2493 2495 2505 2521 2592"
ret = string.replace(" ", ", ")
print(ret)

0, 71, 87, 97, 99, 103, 113, 114, 114, 128, 128, 128, 128, 129, 131, 131, 131, 131, 147, 156, 158, 163, 163, 202, 217, 227, 228, 234, 241, 242, 242, 243, 244, 256, 259, 259, 259, 262, 270, 277, 278, 289, 292, 294, 314, 330, 348, 355, 356, 357, 358, 370, 373, 375, 375, 376, 381, 387, 390, 405, 405, 417, 422, 423, 428, 429, 445, 460, 478, 484, 484, 495, 503, 504, 504, 504, 506, 516, 518, 518, 521, 531, 533, 536, 550, 551, 559, 573, 591, 592, 592, 609, 615, 617, 631, 632, 634, 635, 647, 647, 651, 662, 662, 664, 664, 672, 679, 679, 681, 687, 706, 723, 737, 738, 745, 746, 748, 762, 762, 763, 778, 778, 786, 790, 792, 792, 793, 793, 809, 810, 810, 834, 835, 835, 851, 865, 874, 876, 879, 889, 893, 906, 906, 907, 909, 920, 921, 921, 923, 925, 940, 941, 947, 949, 949, 966, 979, 996, 1005, 1007, 1020, 1020, 1034, 1034, 1037, 1037, 1037, 1040, 1051, 1052, 1054, 1056, 1063, 1068, 1069, 1076, 1077, 1094, 1110, 1135, 1148, 1159, 1163, 1165, 1165, 1165, 1166, 1167, 1167, 1168, 1168, 1168, 1168, 1183, 